In [2]:
try:
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2
    from pymongo import MongoClient
except:
    !pip install pandas numpy faker sqlalchemy psycopg2-binary pymongo
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2
    from pymongo import MongoClient

  Using cached pandas-1.4.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached Faker-13.12.0-py3-none-any.whl (1.6 MB)
  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached pymongo-4.1.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (471 kB)


### Create the CSV data and import to the database as pre-requisite

In [3]:
from faker import Faker
import csv
output=open('data.csv','w')
fake=Faker()
header=['name','age','street','city','state','zip','lng','lat']
mywriter=csv.writer(output)
mywriter.writerow(header)
for r in range(1000):
    row =[fake.name(),fake.random_int(min=18,max=80, step=1), 
                       fake.street_address(), fake.city(),fake.state(),
                       fake.zipcode(),fake.longitude(),fake.latitude()]
    #print(row)
    mywriter.writerow(row)

output.close()

### Create the connection to the database

Due to a bug in the libpq packages for Mac M1 laptops, a workaround is to run the containers with rosetta or on intel based Linux machine
ref link: https://stackoverflow.com/questions/62807717/how-can-i-solve-postgresql-scram-authentifcation-problem

In [4]:
host='ibrahim_postgres'
username='postgres'
password='password123'
database='deproject'
port='5432'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
print(engine.table_names())

[]


/tmp/ipykernel_320/1324543537.py:8: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


### Send the csv data to the postgres database

In [5]:
df = pd.read_csv('./data.csv')
df.head()

,name,age,street,city,state,zip,lng,lat
0,Rebecca Williams,33,5401 White Plaza Suite 415,East Ambertown,Indiana,91940,151.916927,25.865012
1,Jennifer Everett,25,583 Stuart Road Suite 833,Ballardmouth,New York,95720,32.021531,-80.857481
2,Destiny Butler,57,03677 Danielle Point,Kevintown,Utah,79672,164.736352,32.806609
3,Sarah Barron,44,622 Thomas Crossing,New Beverlyberg,Rhode Island,49172,-139.152894,-8.860368
4,Carlos Allen,47,823 Gabriel Brooks,Port Laura,Nebraska,37081,139.062547,82.172667


In [6]:
df.to_sql('data',engine, if_exists='replace', index=False)

1000

### Retrive Data from postgres as first step in the DAG

In [6]:
def PSQL_to_DF():
    host='ibrahim_postgres'
    username='postgres'
    password='password123'
    database='deproject'
    port='5432'
    engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
    
    df = pd.read_sql('SELECT * FROM data', engine)
    
    return df.copy()
    

In [7]:
df = PSQL_to_DF()
df.head()

,name,age,street,city,state,zip,lng,lat
0,Colleen Williams,45,92320 Shaun Hill,Lake Mikeport,Nevada,86538,-146.397166,-10.178631
1,Justin Garrett,65,37772 Kristin Mountain Suite 231,Danielleport,Michigan,10745,-108.015689,-35.436433
2,Jasmine Williams,52,389 Ramirez Terrace Apt. 351,Port Tiffanyton,Arkansas,5351,122.584334,-59.356266
3,John Smith,79,55137 Alice Viaduct,Geraldside,Tennessee,7426,-59.858268,62.087841
4,Jennifer Monroe,35,63078 Silva Knolls Suite 903,Campbellberg,Wyoming,10847,140.823846,60.366340


### Convert DF to json as second task on DAG

In [16]:
def CSV_to_JSON(df):
    json_list = df.to_dict(orient='records')
    return json_list.copy()

In [17]:
json_list = CSV_to_JSON(df)

### Write the converted json list to the Mongo DB as last step

In [18]:
def JSON_to_Mongo(data):
    client = MongoClient('mongo-db:27017', username='root',password='example')
    db = client['de_hw1']
    collection = db.jsonData
    for i in range(0,len(data)):
        try:
            result = collection.insert_one(data[i])
        except:
            print(f'Error for reoord {i}')


In [19]:
JSON_to_Mongo(json_list)

In [22]:
connection = MongoClient('mongo-db:27017', username='root',password='example')
connection.list_database_names()
db_conn = connection['de_hw1']
collection_conn = db_conn.jsonData
print(json_list[0])
cursor = collection_conn.find().limit(10)
for i in cursor:
    print(i)

{'name': 'Colleen Williams', 'age': 45, 'street': '92320 Shaun Hill', 'city': 'Lake Mikeport', 'state': 'Nevada', 'zip': 86538, 'lng': -146.397166, 'lat': -10.178631, '_id': ObjectId('629109c52ca3cecf4c8679d8')}
{'_id': ObjectId('629108d32ca3cecf4c8675ed'), 'name': 'Colleen Williams', 'age': 45, 'street': '92320 Shaun Hill', 'city': 'Lake Mikeport', 'state': 'Nevada', 'zip': 86538, 'lng': -146.397166, 'lat': -10.178631}
{'_id': ObjectId('629108d32ca3cecf4c8675ee'), 'name': 'Justin Garrett', 'age': 65, 'street': '37772 Kristin Mountain Suite 231', 'city': 'Danielleport', 'state': 'Michigan', 'zip': 10745, 'lng': -108.015689, 'lat': -35.4364325}
{'_id': ObjectId('629108d32ca3cecf4c8675ef'), 'name': 'Jasmine Williams', 'age': 52, 'street': '389 Ramirez Terrace Apt. 351', 'city': 'Port Tiffanyton', 'state': 'Arkansas', 'zip': 5351, 'lng': 122.584334, 'lat': -59.356266}
{'_id': ObjectId('629108d32ca3cecf4c8675f0'), 'name': 'John Smith', 'age': 79, 'street': '55137 Alice Viaduct', 'city': 'G

### Airflow implementation

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime,timedelta
try:
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2
    from pymongo import MongoClient
except:
    !pip install pandas numpy faker sqlalchemy psycopg2-binary pymongo
    import pandas as pd
    import numpy as np
    from faker import Faker
    import csv
    from sqlalchemy import create_engine
    import psycopg2
    from pymongo import MongoClient
    
default_args={
    'owner': 'ibrahim',
    'start_date': dt.datetime(2022, 5, 27),
    'retries': 1,
    'retry_delay': dt.timedelta(minutes=5),
}

def PSQL_to_DF():
    host='ibrahim_postgres'
    username='postgres'
    password='password123'
    database='deproject'
    port='5432'
    engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
    
    df = pd.read_sql('SELECT * FROM data', engine)
    
    return df.copy()

def CSV_to_JSON(df):
    df = df.xcom_pull(task_id='Read_from_DB')
    json_list = df.to_dict(orient='records')
    return json_list.copy()

def JSON_to_Mongo(data):
    data = data.xcom_pull(task_id='Convert_to_Json')
    client = MongoClient('mongo-db:27017', username='root',password='example')
    db = client['de_hw1']
    collection = db.jsonData
    for i in range(0,len(data)):
        try:
            result = collection.insert_one(data[i])
        except:
            print(f'Error for reoord {i}')

            
with DAG('HW1_Q2',
         default_args=default_args,
         schedule_interval=timedelta(minutes=1),  
         catchup=False,     
         ) as dag:
    starting_point = BashOperator(task_id='starting_point', bash_command='echo "I am reading the CSV now....."')
    psqlTOdf = PythonOperator(task_id='Read_from_DB',python_callable=PSQL_to_DF)
    csvTOjson = PythonOperator(task_id='Convert_to_Json',python_callable=CSV_to_JSON)
    jsonTOmongo = PythonOperator(task_id='Write_to_Mongo',python_callable=JSON_to_Mongo)
    ending_point = BashOperator(task_id='ending_point', bash_command='echo "I am done with this DAG ^_^"')

starting_point >> psqlTOdf >> csvTOjson >> jsonTOmongo >> ending_point